In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import CountVectorizer
from googletrans import Translator
from tqdm import tqdm
from wordcloud import WordCloud
import matplotlib.pyplot as plt
from nltk import tokenize
import nltk
import seaborn as sns
import unidecode
from string import punctuation

In [ ]:
ds_columns  = ["sentiment", "ids", "date", "flag", "user", "text"]
ds_encoding = "ISO-8859-1"
df_tweet_original = pd.read_csv(r'D:\Estudos\FIAP\Fase 5\Tweets.csv', sep=',', encoding=ds_encoding, names=ds_columns)
df_tweet_original.head()

In [ ]:
df_tweet = df_tweet_original.sample(2000).reset_index(drop=True)

In [ ]:
df_tweet.sentiment.value_counts()

In [ ]:
vetorizar = CountVectorizer(lowercase=False, max_features=2000)
bag_of_words = vetorizar.fit_transform(df_tweet.text)
# print(bag_of_words)
matriz_esparsa = pd.DataFrame.sparse.from_spmatrix(bag_of_words, columns=vetorizar.get_feature_names())

In [ ]:
# matriz_esparsa

In [ ]:
def classificacao_texto(df, coluna_texto, coluna_classificacao):
    vetorizar = CountVectorizer(lowercase=False, max_features=2000)
    bag_of_words = vetorizar.fit_transform(df[coluna_texto])
    # matriz_esparsa = pd.DataFrame.sparse.from_spmatrix(bag_of_words, columns=vetorizar.get_feature_names())
    
    treino, teste, classe_treino, classe_teste = train_test_split(bag_of_words, 
                                                                    df[coluna_classificacao], 
                                                                    random_state=42)
    regressao_logistica = LogisticRegression()
    regressao_logistica.fit(treino,classe_treino)
    return regressao_logistica.score(teste, classe_teste)


In [ ]:
classificacao_texto(df_tweet,'text','sentiment')

In [ ]:
lista_df = list()
for frase in df_tweet.text:
    frase_teste = []
    for palavra in frase.split():
        if '@' in palavra or '.com' in palavra:
            continue
        else:
            frase_teste.append(palavra)
    lista_df.append(' '.join(frase_teste))

df_tweet['text_v2'] = lista_df

In [ ]:
trans = Translator()
frase_traduzida = list()
for frase in tqdm(df_tweet.text_v2):
    try:
        frase_traduzida.append(trans.translate(frase, dest='pt').text)
    except:
        frase_traduzida.append('')
df_tweet['text_pt'] = frase_traduzida

In [ ]:
df_tweet.to_csv('arquivo_traduzido.csv' , index=False)

In [ ]:
classificacao_texto(df_tweet,'text_pt','sentiment')

In [ ]:
def nuvem_palavras(texto, coluna_texto):
    todas_palavras = ' ' .join([texto for texto in texto[coluna_texto]])
    nuvem_palavras = WordCloud(width=400, height=250, max_font_size=50, collocations=False).generate(todas_palavras)
    plt.figure(figsize=(10,7))
    plt.imshow(nuvem_palavras,interpolation='bilinear')
    plt.show()

In [ ]:
nuvem_palavras(df_tweet, 'text_pt')

In [ ]:
todas_palavras = ' ' .join([texto for texto in df_tweet.text_pt])
token_espaco = tokenize.WhitespaceTokenizer()
token_frase = token_espaco.tokenize(todas_palavras)
frequencia = nltk.FreqDist(token_frase)

In [ ]:
df_frequencia = pd.DataFrame({
                    'Palavra':list(frequencia.keys()),
                    'Frequencia':list(frequencia.values())})

In [ ]:
df_frequencia.nlargest(columns='Frequencia',n=10)

In [ ]:
def pareto(texto, coluna_texto, quantidade):
  todas_palavras = ' '.join([resenha for resenha in texto[coluna_texto]])
  token_espaco = nltk.tokenize.WhitespaceTokenizer()
  token_frase = token_espaco.tokenize(todas_palavras)
  frequencias = nltk.FreqDist(token_frase)
  df_frequencias = pd.DataFrame({'Palavras': list(frequencias.keys()),
                               'Frequencia': list(frequencias.values())})
  df_frequencias = df_frequencias.nlargest(n=quantidade, columns='Frequencia')

  total = df_frequencias['Frequencia'].sum()
  df_frequencias['Porcentagem'] = df_frequencias['Frequencia'].cumsum() / total * 100

  plt.figure(figsize=(12,8))
  ax = sns.barplot(data=df_frequencias, x='Palavras', y='Frequencia', color='gray')
  ax2 = ax.twinx()
  sns.lineplot(data=df_frequencias, x='Palavras', y='Porcentagem', color='red', sort=False, ax=ax2)
  plt.show()

In [ ]:
pareto(df_tweet,'text_pt',10)

In [ ]:
palavras_irrelevantes = nltk.corpus.stopwords.words('portuguese')

tweet_processado = list()
for tweet in df_tweet.text_pt:
    novo_tweet = list()
    palavras_texto = token_espaco.tokenize(tweet)
    for palavra in palavras_texto:
        if palavra not in palavras_irrelevantes:
            novo_tweet.append(palavra)
    tweet_processado.append(' '.join(novo_tweet))

df_tweet['text_pt_v2'] = tweet_processado

In [ ]:
df_tweet.head()

In [ ]:
classificacao_texto(df_tweet, 'text_pt_v2', 'sentiment')

In [ ]:
pareto(df_tweet,'text_pt_v2',10)

In [ ]:
token_pontuacao = tokenize.WordPunctTokenizer()
pontuacao = list()
for ponto in punctuation:
    pontuacao.append(ponto)

pontuacao_stopwords = pontuacao + palavras_irrelevantes

tweet_processado = list()
for tweet in df_tweet['text_pt_v2']:
    novo_tweet = list()
    palavras_texto = token_pontuacao.tokenize(tweet)
    for palavra in palavras_texto:
        if palavra not in pontuacao_stopwords:
            novo_tweet.append(palavra)
    tweet_processado.append(' '.join(novo_tweet))

df_tweet['text_pt_v3'] = tweet_processado

In [ ]:
classificacao_texto(df_tweet, 'text_pt_v3', 'sentiment')

In [ ]:
sem_acentos = [unidecode.unidecode(texto) for texto in df_tweet.text_pt_v3]
stopwords = [unidecode.unidecode(texto) for texto in pontuacao_stopwords]

In [ ]:
df_tweet['text_pt_v4'] = sem_acentos
stemmer = nltk.RSLPStemmer()

tweet_processado = list()
for tweet in df_tweet.text_pt_v4:
    novo_tweet = list()
    tweet = tweet.lower()
    palavras_texto = token_pontuacao.tokenize(tweet)
    for palavra in palavras_texto:
        if palavra not in pontuacao_stopwords:
            novo_tweet.append(stemmer.stem(palavra))
    tweet_processado.append(' '.join(novo_tweet))

df_tweet['text_pt_v4'] = tweet_processado

In [ ]:
classificacao_texto(df_tweet, 'text_pt_v4', 'sentiment')

In [ ]:
pareto(df_tweet, 'text_pt_v4',10)

Codigo para teste, ainda preciso entender melhor como implementar.

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

frases_teste = ['Assisti um filme ótimo','Assisti um filme péssivo']

tfidf = TfidfVectorizer(lowercase=False, max_features=50)

caracteristicas = tfidf.fit_transform(frases_teste)
pd.DataFrame(
    caracteristicas.todense(),
    columns=tfidf.get_feature_names()
)


In [ ]:
from nltk import ngrams

frases = 'Assisti um ótimo filme.'
frase_separada = token_espaco.tokenize(frases)
pares = ngrams(frase_separada,2)
list(pares)

In [ ]:
tfidf = TfidfVectorizer(lowercase=False, ngram_range=(1,2))
vetor_tfidf = tfidf.fit_transform(df_tweet['text_pt'])
treino, teste, classe_treino,classe_teste = train_test_split(vetor_tfidf,
                                                            df_tweet['sentiment'],
                                                            random_state=42)
regressao_logistica = LogisticRegression()
regressao_logistica.fit(treino,classe_treino)
regressao_logistica.score(teste, classe_teste)

In [ ]:
tfidf = TfidfVectorizer(lowercase=False, ngram_range=(1,2))
vetor_tfidf = tfidf.fit_transform(df_tweet['text_pt_v4'])
treino, teste, classe_treino,classe_teste = train_test_split(vetor_tfidf,
                                                            df_tweet['sentiment'],
                                                            random_state=42)
regressao_logistica = LogisticRegression()
regressao_logistica.fit(treino,classe_treino)
regressao_logistica.score(teste, classe_teste)